In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Log\tuning.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Feature_Selection\Filtered_Featuress.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0.1        8493 non-null   int64  
 1   Unnamed: 0          8493 non-null   int64  
 2   Video ID            8493 non-null   int64  
 3   Video Title         8493 non-null   int64  
 4   Channel             8493 non-null   int64  
 5   Published Date      8493 non-null   float64
 6   Views               8493 non-null   int64  
 7   Likes               8493 non-null   int64  
 8   Comments            8493 non-null   int64  
 9   Subscribers         8493 non-null   int64  
 10  Channel Views       8493 non-null   int64  
 11  Country             8493 non-null   int64  
 12  Region              8493 non-null   int64  
 13  CategoryID          8493 non-null   int64  
 14  CategoryName        8493 non-null   int64  
 15  Cluster             8493 non-null   int64  
 16  Catego

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# X va y ajratamiz
X = df.drop('CategoryName', axis=1)
y = df['CategoryName']

# Train / Validation / Test (60/20/20 bo‘linadi)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Decision Tree Regressor model
dt = RandomForestClassifier(random_state=42)
dt.fit(X_train, y_train)

# Baholash (Validation)
y_pred = dt.predict(X_val)

# Natijalar
print("📊 Accuracy", accuracy_score(y_val, y_pred))
print("📉 Classification Report", classification_report(y_val, y_pred))


📊 Accuracy 1.0
📉 Classification Report               precision    recall  f1-score   support

           0       1.00      1.00      1.00       208
           1       1.00      1.00      1.00        61
           2       1.00      1.00      1.00       273
           3       1.00      1.00      1.00       118
           4       1.00      1.00      1.00       118
           5       1.00      1.00      1.00       158
           6       1.00      1.00      1.00       131
           7       1.00      1.00      1.00       142
           8       1.00      1.00      1.00       194
           9       1.00      1.00      1.00       182
          10       1.00      1.00      1.00       114

    accuracy                           1.00      1699
   macro avg       1.00      1.00      1.00      1699
weighted avg       1.00      1.00      1.00      1699



In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 🔹 Model yaratish
rf = RandomForestClassifier(random_state=42)

# 🔹 Parametrlar to‘plami (tuning uchun)
param_grid = {
    'n_estimators': [50, 100, 200, 300],            # Daraxtlar soni
    'max_depth': [None, 5, 10, 15, 20],             # Daraxt chuqurligi
    'min_samples_split': [2, 5, 10],                # Split qilish uchun minimal namunalar
    'min_samples_leaf': [1, 2, 4],                  # Leaf dagi minimal namunalar soni
    'max_features': ['sqrt', 'log2', None],         # Har splitda ko‘rib chiqiladigan xususiyatlar
    'bootstrap': [True, False]                      # Bootstrap namunalarini ishlatish yoki yo‘q
}

# 🔹 GridSearchCV sozlash (5-fold cross-validation)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',    # classification uchun
    n_jobs=-1,
    verbose=2
)

# 🔹 Modelni o‘qitish
grid_search.fit(X_train, y_train)

# 🔹 Eng yaxshi parametrlar
print("✅ Eng yaxshi parametrlar:", grid_search.best_params_)

# 🔹 Eng yaxshi modelni olish
best_rf = grid_search.best_estimator_

# 🔹 Test to‘plamida baholash
y_pred = best_rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("\n🎯 Test Accuracy:", round(accuracy, 4))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# 🎯 Parametrlar diapazoni (tasodifiy kombinatsiyalar sinov uchun)
param_dist = {
    'n_estimators': [50, 100, 150, 200, 300],            # Daraxtlar soni
    'max_depth': [3, 5, 7, 10, 15, 20, None],            # Chuqurlik
    'min_samples_split': [2, 5, 10, 15],                 # Bo‘linish uchun minimal namunalar
    'min_samples_leaf': [1, 2, 4, 6],                    # Leaf node dagi minimal namunalar
    'max_features': ['sqrt', 'log2', None],              # Tanlanadigan xususiyatlar soni
    'bootstrap': [True, False],                          # Bootstrap namunalar ishlatiladimi yoki yo‘q
    'criterion': ['gini', 'entropy', 'log_loss']         # Split kriteriylari
}

# 🔹 Model yaratish
rf_clf = RandomForestClassifier(random_state=42)

# 🔹 RandomizedSearchCV sozlash
random_search = RandomizedSearchCV(
    estimator=rf_clf,
    param_distributions=param_dist,
    n_iter=50,                 # 50 ta tasodifiy kombinatsiyani sinovdan o‘tkazadi
    scoring='accuracy',        # Accuracy asosida baholash
    cv=5,
